In [7]:
import pickle
import pandas as pds
import datetime

%load_ext autoreload

%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [16]:
file = pickle.load(open('py3dcore_h4c/custom_data/solo_2022sep.p', 'rb'))
data = pds.DataFrame(file)
#data['time'] = pds.to_datetime(data['time'])
#data.set_index('time',  inplace=True)
#data.index.name = None
#data.index = data.index.tz_localize(None)

In [19]:
data

,time,bx,by,bz,bt,r,lat,lon,x,y,z,vx,vy,vz,vt,tp,np
0,2022-08-31 00:00:00,0.029020,-16.435475,-8.135658,18.338882,0.757695,-2.879138,147.687434,0.196481,0.006403,-0.731749,609.740601,-52.652805,68.371964,615.817053,3.162292e+05,10.865103
1,2022-08-31 00:01:00,0.029020,-16.435475,-8.135658,18.338882,0.757689,-2.879186,147.687591,0.196444,0.006433,-0.731752,609.740601,-52.652805,68.371964,615.817053,3.162292e+05,10.865103
2,2022-08-31 00:02:00,0.029020,-16.435475,-8.135658,18.338882,0.757682,-2.879233,147.687748,0.196406,0.006463,-0.731755,609.740601,-52.652805,68.371964,615.817053,3.162292e+05,10.865103
3,2022-08-31 00:03:00,0.029020,-16.435475,-8.135658,18.338882,0.757676,-2.879280,147.687905,0.196369,0.006492,-0.731758,609.740601,-52.652805,68.371964,615.817053,3.162292e+05,10.865103
4,2022-08-31 00:04:00,0.758405,-15.021704,-8.239454,17.149792,0.757670,-2.879327,147.688062,0.196332,0.006522,-0.731761,609.740601,-52.652805,68.371964,615.817053,3.162292e+05,10.865103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18715,2022-09-12 23:55:00,9.965065,-4.377737,-0.034102,10.884313,0.624461,-5.028653,153.133238,-0.411440,0.427724,0.194219,591.793396,13.568009,-51.051800,594.146279,4.328101e+41,14.493595
18716,2022-09-12 23:56:00,9.458576,-4.851109,-0.793987,10.659660,0.624453,-5.028811,153.133729,-0.411623,0.427495,0.194310,591.793396,13.568009,-51.051800,594.146279,4.328101e+41,14.493595
18717,2022-09-12 23:57:00,8.093511,-5.018920,-1.481281,9.637877,0.624445,-5.028969,153.134219,-0.411806,0.427266,0.194401,591.793396,13.568009,-51.051800,594.146279,4.328101e+41,14.493595
18718,2022-09-12 23:58:00,6.810724,-5.369940,-2.262874,8.963415,0.624437,-5.029126,153.134709,-0.411989,0.427036,0.194492,591.793396,13.568009,-51.051800,594.146279,4.328101e+41,14.493595


In [10]:
import py3dcore_h4c

fitter = py3dcore_h4c.ABC_SMC()

model_kwargs = {
        "ensemble_size": int(2**17), #2**17
        "iparams": {
           "cme_longitude": {
               "maximum": -110,
               "minimum": -130
           },
           "cme_latitude": {
               "maximum": -10,
               "minimum": -50
           },
           "cme_inclination": {
               "maximum": 10,
               "minimum": 0
           }, 
            "cme_launch_velocity": {
                "maximum": 750,
                "minimum": 250
            },
            "cme_launch_radius": {
                "maximum": 30,
                "minimum": 5
            }
        }
    }

t_launch = datetime.datetime(2022, 9, 5, 18, 45, tzinfo=datetime.timezone.utc)

t_s_psp = datetime.datetime(2022, 9, 7, 0, 30, tzinfo=datetime.timezone.utc)
t_e_psp = datetime.datetime(2022, 9, 8, 5, 0,  tzinfo=datetime.timezone.utc)

t_psp = [
    datetime.datetime(2022, 9, 7, 2, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 9, 7, 4, tzinfo=datetime.timezone.utc) ,
    datetime.datetime(2022, 9, 7, 6, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 9, 7, 8, tzinfo=datetime.timezone.utc),
]
fitter.initialize(t_launch, py3dcore_h4c.ToroidalModel, model_kwargs)
fitter.add_observer("SOLO", t_psp, t_s_psp, t_e_psp)

In [11]:
dt_all = [t_s_psp] + t_psp + [t_e_psp]
from py3dcore_h4c.fitter.base import custom_observer

observer_obj = custom_observer('solo_2022_sep_mag_ll_swa_science.p')

t, data = observer_obj.get([t_s_psp, t_e_psp], "mag", reference_frame= 'HEEQ', sampling_freq=300, use_cache=True, as_endpoints=True)
tra = observer_obj.trajectory(dt_all, reference_frame='HEEQ', data_key = "pos")

In [14]:
tra

array([[-0.59489415,  0.34503881, -0.04499631],
       [-0.59448726,  0.34445327, -0.04510713],
       [-0.59394445,  0.34366993, -0.04525464],
       [-0.59340131,  0.34288361, -0.04540187],
       [-0.59285783,  0.34209429, -0.0455488 ],
       [-0.58713015,  0.33362425, -0.0470737 ]])

In [9]:
import heliosat
t_launch = datetime.datetime(2022, 6, 2, 6, tzinfo=datetime.timezone.utc)

t_s_psp = datetime.datetime(2022, 6, 2, 10, tzinfo=datetime.timezone.utc)
t_e_psp = datetime.datetime(2022, 6, 3, 6, tzinfo=datetime.timezone.utc)

t_psp = [datetime.datetime(2022, 6, 2, 13, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 6, 2, 14, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 6, 2, 15, tzinfo=datetime.timezone.utc),
    datetime.datetime(2022, 6, 2, 16, tzinfo=datetime.timezone.utc)]

psp_t, psp_b = heliosat.PSP().get([t_s_psp, t_e_psp], "mag", reference_frame="HEEQ", as_endpoints=True, return_datetimes=True, smoothing="gaussian")


The history saving thread hit an unexpected error (OperationalError('disk I/O error')).History will not be written to the database.


In [10]:
psp_t

array([datetime.datetime(2022, 6, 2, 10, 0, tzinfo=datetime.timezone.utc),
       datetime.datetime(2022, 6, 2, 10, 1, 0, 50042, tzinfo=datetime.timezone.utc),
       datetime.datetime(2022, 6, 2, 10, 2, 0, 100083, tzinfo=datetime.timezone.utc),
       ...,
       datetime.datetime(2022, 6, 3, 5, 57, 59, 899917, tzinfo=datetime.timezone.utc),
       datetime.datetime(2022, 6, 3, 5, 58, 59, 949958, tzinfo=datetime.timezone.utc),
       datetime.datetime(2022, 6, 3, 6, 0, tzinfo=datetime.timezone.utc)],
      dtype=object)

In [12]:
from py3dcore_h4c.fitter.base import CustomData

data_obj = CustomData(fitter.observers, 'HEEQ','solo_2022sep.p')
data_obj.generate_noise("psd", 300)

ValueError: could not broadcast input array from shape (342,3) into shape (342,)